In [ ]:
!pip install gradio>=4.0.0 pandas>=1.5.0 plotly>=5.15.0 textblob>=0.17.1 numpy>=1.21.0 python-dateutil>=2.8.2

In [ ]:
!pip install transformers>=4.35.0 torch>=2.0.0 accelerate>=0.20.0 bitsandbytes>=0.41.0


In [ ]:
import gradio as gr
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import random
from textblob import TextBlob
import time
import os
from typing import Dict, List, Tuple
import uuid
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import gc

# Step 3: Download and initialize the IBM Granite model
print("🔄 Downloading IBM Granite 3.3-2B model...")
print("⏳ This may take a few minutes on first run...")

MODEL_NAME = "ibm-granite/granite-3.0-2b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"

try:
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Load model with optimization for Colab
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        load_in_8bit=True if torch.cuda.is_available() else False,
        trust_remote_code=True
    )

    print(f"✅ Model loaded successfully on {device}")
    model_loaded = True

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("🔄 Falling back to CPU with basic settings...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float32,
            trust_remote_code=True
        )
        model_loaded = True
        print("✅ Model loaded successfully on CPU")
    except:
        model_loaded = False
        print("❌ Failed to load model. Some features may not work.")

# Global data storage (in production, use proper database)
user_data = {}
complaints = []
feedback_data = []
notifications = []
sentiment_history = []
chat_history = []

# Step 4: Define the CitizenAI class with local model
class CitizenAI:
    def __init__(self):
        self.model_loaded = model_loaded
        if self.model_loaded:
            self.tokenizer = tokenizer
            self.model = model
        self.initialize_sample_data()

    def initialize_sample_data(self):
        """Initialize with sample data for demonstration"""
        global feedback_data, sentiment_history, notifications, complaints

        # Sample feedback data
        sample_feedback = [
            "Water supply has improved significantly in our area",
            "Road conditions are terrible, need immediate attention",
            "The new online portal is very user-friendly",
            "Garbage collection is irregular in our locality",
            "Government hospital services have deteriorated"
        ]

        for feedback in sample_feedback:
            sentiment = self.analyze_sentiment(feedback)
            feedback_data.append({
                'text': feedback,
                'sentiment': sentiment,
                'timestamp': datetime.now() - timedelta(days=random.randint(1, 30))
            })
            sentiment_history.append({
                'date': datetime.now() - timedelta(days=random.randint(1, 30)),
                'sentiment': sentiment
            })

        # Sample notifications
        notifications.extend([
            {"title": "Water Supply Disruption", "message": "Water supply will be disrupted tomorrow from 10 AM to 4 PM", "type": "alert"},
            {"title": "New Scheme Launch", "message": "PM Kisan Samman Nidhi registration is now open", "type": "info"},
            {"title": "Road Closure", "message": "Main Street will be closed for repairs this weekend", "type": "warning"}
        ])

        # Sample complaints
        complaints.extend([
            {"id": "CMP001", "category": "Roads", "status": "In Progress", "description": "Pothole on Main Street"},
            {"id": "CMP002", "category": "Water", "status": "Resolved", "description": "Low water pressure"},
            {"id": "CMP003", "category": "Sanitation", "status": "Pending", "description": "Irregular garbage collection"}
        ])

    def generate_with_local_model(self, prompt: str, max_length: int = 200) -> str:
        """Generate text using the local IBM Granite model"""
        if not self.model_loaded:
            return "AI model is not available. Please try again later."

        try:
            # Format the prompt for the model
            formatted_prompt = f"<|user|>\n{prompt}\n<|assistant|>\n"

            # Tokenize input
            inputs = self.tokenizer.encode(formatted_prompt, return_tensors="pt")
            if torch.cuda.is_available():
                inputs = inputs.to(device)

            # Generate response
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_new_tokens=max_length,
                    temperature=0.7,
                    do_sample=True,
                    top_p=0.9,
                    pad_token_id=self.tokenizer.eos_token_id,
                    repetition_penalty=1.1
                )

            # Decode and clean response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = response.replace(formatted_prompt, "").strip()

            # Clean up GPU memory
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            return response if response else "I'm processing your request. Please try again."

        except Exception as e:
            print(f"Generation error: {e}")
            return "I'm experiencing technical difficulties. Please try again."

    def analyze_sentiment(self, text: str) -> str:
        """Analyze sentiment of given text"""
        try:
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity

            if polarity > 0.1:
                return "Positive"
            elif polarity < -0.1:
                return "Negative"
            else:
                return "Neutral"
        except:
            return "Neutral"

    def conversational_ai_assistant(self, message: str, history: List[Tuple[str, str]]) -> Tuple[str, List[Tuple[str, str]]]:
        """Main conversational AI assistant"""
        if not message.strip():
            return "", history

        # Create context-aware prompt
        context_prompt = f"""You are a helpful AI assistant for a citizen engagement platform in India.
You help citizens with government services, schemes, complaints, and general civic queries.
Provide helpful, accurate, and contextual responses about Indian government services.

Citizen Query: {message}

Please provide a helpful response:"""

        # Get response from local model
        ai_response = self.generate_with_local_model(context_prompt, max_length=250)

        # Update history
        history.append((message, ai_response))

        # Store in chat history for analytics
        chat_history.append({
            'user_message': message,
            'ai_response': ai_response,
            'timestamp': datetime.now()
        })

        return "", history

    def submit_feedback(self, feedback_text: str) -> str:
        """Submit and analyze citizen feedback"""
        if not feedback_text.strip():
            return "Please enter your feedback."

        sentiment = self.analyze_sentiment(feedback_text)

        feedback_entry = {
            'text': feedback_text,
            'sentiment': sentiment,
            'timestamp': datetime.now()
        }

        feedback_data.append(feedback_entry)
        sentiment_history.append({
            'date': datetime.now(),
            'sentiment': sentiment
        })

        return f"Thank you for your feedback! Sentiment detected: {sentiment}"

    def register_complaint(self, category: str, description: str, contact: str) -> str:
        """Register a new complaint"""
        if not description.strip():
            return "Please provide a description of your complaint."

        complaint_id = f"CMP{len(complaints) + 1:03d}"

        complaint = {
            'id': complaint_id,
            'category': category,
            'description': description,
            'contact': contact,
            'status': 'Registered',
            'timestamp': datetime.now()
        }

        complaints.append(complaint)

        return f"Complaint registered successfully! Your complaint ID is: {complaint_id}"

    def create_sentiment_dashboard(self):
        """Create sentiment analysis dashboard"""
        if not sentiment_history:
            fig = go.Figure()
            fig.add_annotation(text="No sentiment data available", xref="paper", yref="paper", x=0.5, y=0.5)
            return fig

        # Count sentiments
        sentiment_counts = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
        for entry in sentiment_history:
            sentiment_counts[entry['sentiment']] += 1

        # Create pie chart
        fig = px.pie(
            values=list(sentiment_counts.values()),
            names=list(sentiment_counts.keys()),
            title="Citizen Sentiment Distribution",
            color_discrete_map={
                'Positive': '#28a745',
                'Negative': '#dc3545',
                'Neutral': '#ffc107'
            }
        )

        return fig

    def create_complaint_status_chart(self):
        """Create complaint status visualization"""
        if not complaints:
            fig = go.Figure()
            fig.add_annotation(text="No complaint data available", xref="paper", yref="paper", x=0.5, y=0.5)
            return fig

        status_counts = {}
        for complaint in complaints:
            status = complaint['status']
            status_counts[status] = status_counts.get(status, 0) + 1

        fig = px.bar(
            x=list(status_counts.keys()),
            y=list(status_counts.values()),
            title="Complaint Status Overview",
            labels={'x': 'Status', 'y': 'Count'}
        )

        return fig

    def get_user_profile(self, name: str, citizen_id: str, region: str) -> str:
        """Create/update user profile"""
        if not name or not citizen_id:
            return "Please provide both name and citizen ID."

        user_data[citizen_id] = {
            'name': name,
            'region': region,
            'created_at': datetime.now()
        }

        return f"Profile created/updated for {name} (ID: {citizen_id}) in {region}"

    def get_notifications(self) -> str:
        """Get latest notifications"""
        if not notifications:
            return "No new notifications."

        notification_text = "📢 Latest Notifications:\n\n"
        for notif in notifications[-5:]:  # Show last 5 notifications
            icon = "🚨" if notif['type'] == 'alert' else "ℹ️" if notif['type'] == 'info' else "⚠️"
            notification_text += f"{icon} {notif['title']}: {notif['message']}\n\n"

        return notification_text

    def generate_policy_summary(self, policy_text: str) -> str:
        """Generate AI-powered policy summary"""
        if not policy_text.strip():
            return "Please enter policy text to summarize."

        prompt = f"""Please provide a simple, citizen-friendly summary of this government policy:

{policy_text[:1000]}...

Create a brief summary highlighting:
1. Main purpose of the policy
2. Who it affects
3. Key benefits for citizens
4. What actions citizens need to take

Keep it simple and easy to understand."""

        summary = self.generate_with_local_model(prompt, max_length=250)
        return f"📋 Policy Summary:\n\n{summary}"

    def get_service_guide(self, service_type: str) -> str:
        """Provide step-by-step service guidance"""
        service_guides = {
            "Ration Card": """📋 Ration Card Application Process:

1. Visit your nearest Common Service Center (CSC) or Tehsil office
2. Required documents:
   - Aadhaar Card
   - Address Proof
   - Income Certificate
   - Family Photo
3. Fill out the application form
4. Submit documents and pay fees (₹15-30)
5. Get acknowledgment receipt
6. Verification will be done within 15 days
7. Collect your ration card

Contact: Local Food & Supply Department""",

            "Birth Certificate": """📋 Birth Certificate Application:

1. Visit Registrar's office within 21 days of birth
2. Required documents:
   - Hospital discharge certificate
   - Parents' ID proof
   - Address proof
3. Fill Form 1 (for registration)
4. No fees if applied within 21 days
5. Late registration (after 21 days): ₹10 fees + affidavit
6. Certificate issued immediately

Contact: Local Registrar Office""",

            "Pension": """📋 Pension Scheme Application:

1. Identify eligible scheme (Old Age/Widow/Disability)
2. Visit Block Development Office or online portal
3. Required documents:
   - Age proof (60+ for old age pension)
   - Income certificate
   - Bank account details
   - Aadhaar Card
4. Fill application form
5. Submit documents
6. Verification by concerned officer
7. Approval and pension starts

Monthly Amount: ₹200-1000 depending on scheme""",

            "Voter ID": """📋 Voter ID Card Application:

1. Visit nearest Electoral Registration Office or apply online
2. Required documents:
   - Age proof (18+ years)
   - Address proof
   - Recent passport photo
3. Fill Form 6 for new registration
4. Submit application with documents
5. Field verification by Booth Level Officer
6. Objection period (7 days)
7. Card issued after verification

Contact: Chief Electoral Officer""",

            "PAN Card": """📋 PAN Card Application:

1. Apply online at NSDL/UTIITSL website
2. Required documents:
   - Identity proof
   - Address proof
   - Date of birth proof
3. Fill Form 49A (Indian citizens)
4. Pay fees (₹107 online, ₹110 physical)
5. Submit documents at nearest center
6. Acknowledgment receipt provided
7. PAN card dispatched within 15 days

Contact: Income Tax Department""",

            "Passport": """📋 Passport Application:

1. Register on Passport Seva Portal
2. Fill online application form
3. Pay fees online (₹1500 for 36 pages)
4. Book appointment at PSK/POPSK
5. Required documents:
   - Address proof
   - Identity proof
   - Date of birth proof
6. Visit PSK on appointment date
7. Police verification (if required)
8. Passport dispatch within 30 days

Contact: Passport Seva Kendra"""
        }

        return service_guides.get(service_type, "Service guide not available. Please contact local authorities.")

    def generate_document_draft(self, doc_type: str, details: str) -> str:
        """Generate AI-powered document drafts using local model"""
        if not details.strip():
            return "Please provide details for document generation."

        if doc_type == "RTI Application":
            prompt = f"""Create a formal Right to Information (RTI) application in India format.

Details provided: {details}

Generate a complete RTI application with:
- Proper addressing format
- Subject line
- Information being sought
- Reason for seeking information
- Contact details placeholder
- Signature section

Make it formal and legally appropriate."""

        elif doc_type == "Complaint Letter":
            prompt = f"""Create a formal complaint letter to government authorities in India.

Issue details: {details}

Generate a complete complaint letter with:
- Date and proper addressing
- Clear subject line
- Respectful salutation
- Problem description
- Expected resolution
- Contact details placeholder
- Formal closing

Keep it professional and effective."""

        elif doc_type == "Application Form":
            prompt = f"""Create a formal application letter for government services in India.

Purpose and details: {details}

Generate a complete application with:
- Date and addressing format
- Clear subject line
- Respectful language
- Purpose statement
- Supporting reasons
- Request for consideration
- Contact details placeholder
- Proper closing

Make it formal and appropriate."""

        else:
            return "Document type not supported."

        try:
            generated_doc = self.generate_with_local_model(prompt, max_length=400)
            return f"📄 Generated {doc_type}:\n\n{generated_doc}\n\n---\nNote: Please review and customize the document before submission."
        except Exception as e:
            return f"Error generating document: {str(e)}. Please try again with simpler details."

    def cleanup_memory(self):
        """Clean up GPU memory periodically"""
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            gc.collect()

# Step 5: Initialize the CitizenAI system
print("🚀 Initializing Citizen AI Platform...")
citizen_ai = CitizenAI()
print("✅ System initialized successfully!")

# Step 6: Create the main Gradio interface
def create_main_interface():
    """Create the main Gradio interface"""

    with gr.Blocks(title="Citizen AI - Intelligent Citizen Engagement Platform", theme=gr.themes.Soft()) as app:

        # Header
        gr.HTML(f"""
        <div style="text-align: center; padding: 20px; background: linear-gradient(90deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 10px; margin-bottom: 20px;">
            <h1>🏛️ Citizen AI - Intelligent Citizen Engagement Platform</h1>
            <p>Your AI-powered gateway to government services and civic engagement</p>
            <p><small>🤖 AI Model: {'✅ IBM Granite 3.0-2B (Local)' if citizen_ai.model_loaded else '❌ Model Not Available'}</small></p>
        </div>
        """)

        with gr.Tabs():
            # Tab 1: AI Assistant
            with gr.Tab("🤖 AI Assistant"):
                gr.Markdown("### Real-Time Conversational AI Assistant")
                gr.Markdown("Ask questions about government services, schemes, complaints, or any civic queries.")

                if not citizen_ai.model_loaded:
                    gr.Markdown("⚠️ **AI Model not loaded.** Some features may be limited.")

                chatbot = gr.Chatbot(
                    height=400,
                    placeholder="Hi! I'm your AI assistant for government services. How can I help you today?"
                )

                with gr.Row():
                    msg = gr.Textbox(
                        placeholder="Type your question here...",
                        label="Your Message",
                        scale=4
                    )
                    send_btn = gr.Button("Send", variant="primary", scale=1)

                # Memory cleanup button
                cleanup_btn = gr.Button("🧹 Clear Memory", variant="secondary", size="sm")

                msg.submit(citizen_ai.conversational_ai_assistant, [msg, chatbot], [msg, chatbot])
                send_btn.click(citizen_ai.conversational_ai_assistant, [msg, chatbot], [msg, chatbot])
                cleanup_btn.click(lambda: citizen_ai.cleanup_memory())

            # Tab 2: Citizen Dashboard
            with gr.Tab("👤 Citizen Dashboard"):
                gr.Markdown("### Personalized Citizen Dashboard")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### Profile Information")
                        name_input = gr.Textbox(label="Full Name")
                        citizen_id = gr.Textbox(label="Citizen ID/Aadhaar")
                        region = gr.Dropdown(
                            choices=["Andhra Pradesh","Delhi", "Mumbai", "Bangalore", "Chennai", "Hyderabad", "Pune", "Kolkata"],
                            label="Region"
                        )
                        profile_btn = gr.Button("Update Profile", variant="primary")
                        profile_status = gr.Textbox(label="Status", interactive=False)

                    with gr.Column():
                        gr.Markdown("#### Notifications & Alerts")
                        notifications_display = gr.Textbox(
                            value=citizen_ai.get_notifications(),
                            label="Latest Notifications",
                            lines=10,
                            interactive=False
                        )
                        refresh_notif = gr.Button("Refresh Notifications")

                profile_btn.click(
                    citizen_ai.get_user_profile,
                    [name_input, citizen_id, region],
                    profile_status
                )
                refresh_notif.click(
                    citizen_ai.get_notifications,
                    outputs=notifications_display
                )

            # Tab 3: Feedback & Complaints
            with gr.Tab("📝 Feedback & Complaints"):
                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### Submit Feedback")
                        feedback_text = gr.Textbox(
                            label="Your Feedback",
                            placeholder="Share your experience with government services...",
                            lines=4
                        )
                        feedback_btn = gr.Button("Submit Feedback", variant="primary")
                        feedback_result = gr.Textbox(label="Result", interactive=False)

                    with gr.Column():
                        gr.Markdown("#### Register Complaint")
                        complaint_category = gr.Dropdown(
                            choices=["Roads", "Water Supply", "Electricity", "Sanitation", "Healthcare", "Education", "Other"],
                            label="Category"
                        )
                        complaint_desc = gr.Textbox(
                            label="Description",
                            placeholder="Describe your complaint in detail...",
                            lines=4
                        )
                        contact_info = gr.Textbox(label="Contact Information")
                        complaint_btn = gr.Button("Register Complaint", variant="primary")
                        complaint_result = gr.Textbox(label="Result", interactive=False)

                feedback_btn.click(
                    citizen_ai.submit_feedback,
                    feedback_text,
                    feedback_result
                )

                complaint_btn.click(
                    citizen_ai.register_complaint,
                    [complaint_category, complaint_desc, contact_info],
                    complaint_result
                )

            # Tab 4: Analytics Dashboard
            with gr.Tab("📊 Analytics Dashboard"):
                gr.Markdown("### Citizen Sentiment & Complaint Analytics")

                with gr.Row():
                    sentiment_plot = gr.Plot(
                        value=citizen_ai.create_sentiment_dashboard(),
                        label="Sentiment Analysis"
                    )
                    complaint_plot = gr.Plot(
                        value=citizen_ai.create_complaint_status_chart(),
                        label="Complaint Status"
                    )

                refresh_dashboard = gr.Button("Refresh Dashboard", variant="secondary")

                def refresh_plots():
                    return citizen_ai.create_sentiment_dashboard(), citizen_ai.create_complaint_status_chart()

                refresh_dashboard.click(
                    refresh_plots,
                    outputs=[sentiment_plot, complaint_plot]
                )

            # Tab 5: Services Guide
            with gr.Tab("🏛️ Services Guide"):
                gr.Markdown("### Government Services Guide")

                service_type = gr.Dropdown(
                    choices=["Ration Card", "Birth Certificate", "Pension", "Voter ID", "PAN Card", "Passport"],
                    label="Select Service"
                )

                get_guide_btn = gr.Button("Get Step-by-Step Guide", variant="primary")
                service_guide = gr.Textbox(
                    label="Service Guide",
                    lines=15,
                    interactive=False
                )

                get_guide_btn.click(
                    citizen_ai.get_service_guide,
                    service_type,
                    service_guide
                )

            # Tab 6: Policy Summarizer
            with gr.Tab("📋 Policy Summarizer"):
                gr.Markdown("### AI-Powered Policy & Document Summarizer")
                gr.Markdown("Paste any government policy, circular, or document to get a simple summary.")

                if not citizen_ai.model_loaded:
                    gr.Markdown("⚠️ **AI Model required for this feature.**")

                policy_input = gr.Textbox(
                    label="Policy/Document Text",
                    placeholder="Paste the policy document or text here...",
                    lines=8
                )

                summarize_btn = gr.Button("Generate Summary", variant="primary", interactive=citizen_ai.model_loaded)

                policy_summary = gr.Textbox(
                    label="AI-Generated Summary",
                    lines=10,
                    interactive=False
                )

                summarize_btn.click(
                    citizen_ai.generate_policy_summary,
                    policy_input,
                    policy_summary
                )

            # Tab 7: Document Generator
            with gr.Tab("📄 Document Generator"):
                gr.Markdown("### AI-Powered Document Generator")
                gr.Markdown("Generate RTI applications, complaint letters, and application forms automatically.")

                if not citizen_ai.model_loaded:
                    gr.Markdown("⚠️ **AI Model required for this feature.**")
                else:
                    gr.Markdown("✅ **Local AI Model Ready** - Generate documents offline!")

                doc_type = gr.Dropdown(
                    choices=["RTI Application", "Complaint Letter", "Application Form"],
                    label="Document Type"
                )

                doc_details = gr.Textbox(
                    label="Document Details",
                    placeholder="Provide details for document generation (e.g., what information you need for RTI, what complaint to make, etc.)",
                    lines=6
                )

                generate_doc_btn = gr.Button("Generate Document", variant="primary", interactive=citizen_ai.model_loaded)

                generated_document = gr.Textbox(
                    label="Generated Document",
                    lines=15,
                    interactive=False
                )

                generate_doc_btn.click(
                    citizen_ai.generate_document_draft,
                    [doc_type, doc_details],
                    generated_document
                )

            # Tab 8: Participatory Governance
            with gr.Tab("🗳️ Participatory Governance"):
                gr.Markdown("### Citizen Participation & Polling")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### Public Polling")
                        poll_question = gr.Textbox(
                            value="Which park development project should be prioritized?",
                            label="Poll Question",
                            interactive=False
                        )

                        poll_options = gr.CheckboxGroup(
                            choices=["Children's Play Area", "Walking Track", "Garden & Landscaping", "Sports Facilities"],
                            label="Select your preference(s)"
                        )

                        vote_btn = gr.Button("Submit Vote", variant="primary")
                        vote_result = gr.Textbox(label="Result", interactive=False)

                        def submit_vote(options):
                            if options:
                                return f"Thank you for voting! You selected: {', '.join(options)}"
                            return "Please select at least one option."

                        vote_btn.click(submit_vote, poll_options, vote_result)

                    with gr.Column():
                        gr.Markdown("#### Public Discussion Forum")
                        discussion_topic = gr.Textbox(
                            value="Current Topic: Traffic Management in City Center",
                            label="Discussion Topic",
                            interactive=False
                        )

                        user_comment = gr.Textbox(
                            label="Your Comment",
                            placeholder="Share your thoughts on this topic...",
                            lines=4
                        )

                        post_comment_btn = gr.Button("Post Comment", variant="primary")
                        comment_result = gr.Textbox(label="Status", interactive=False)

                        def post_comment(comment):
                            if comment.strip():
                                return "Comment posted successfully! It will be reviewed and published shortly."
                            return "Please enter a comment."

                        post_comment_btn.click(post_comment, user_comment, comment_result)

            # Tab 9: Admin Panel
            with gr.Tab("⚙️ Admin Panel"):
                gr.Markdown("### Government Admin Dashboard")
                gr.Markdown("*For government officials to monitor citizen engagement*")

                with gr.Row():
                    with gr.Column():
                        gr.Markdown("#### System Statistics")
                        stats = gr.HTML(f"""
                        <div style="padding: 20px; background: #f8f9fa; border-radius: 8px;">
                            <h4>Platform Statistics</h4>
                            <p>📊 Total Feedback: {len(feedback_data)}</p>
                            <p>📝 Total Complaints: {len(complaints)}</p>
                            <p>💬 Chat Interactions: {len(chat_history)}</p>
                            <p>👥 Active Users: {len(user_data)}</p>
                            <p>🤖 AI Model: {'✅ Loaded' if citizen_ai.model_loaded else '❌ Not Available'}</p>
                        </div>
                        """)

                    with gr.Column():
                        gr.Markdown("#### Quick Actions")
                        broadcast_msg = gr.Textbox(label="Broadcast Message")
                        broadcast_type = gr.Dropdown(
                            choices=["info", "alert", "warning"],
                            value="info",
                            label="Message Type"
                        )
                        broadcast_btn = gr.Button("Send Notification", variant="primary")

                        def send_notification(message, msg_type):
                            if message:
                                notifications.append({
                                    "title": "System Notification",
                                    "message": message,
                                    "type": msg_type
                                })
                                return "Notification sent successfully!"
                            return "Please enter a message."

                        broadcast_result = gr.Textbox(label="Status", interactive=False)
                        broadcast_btn.click(
                            send_notification,
                            [broadcast_msg, broadcast_type],
                            broadcast_result
                        )

                # Complaint Management Section
                with gr.Row():
                    gr.Markdown("#### Complaint Management")

                complaint_df = pd.DataFrame(complaints) if complaints else pd.DataFrame(columns=['id', 'category', 'status', 'description'])
                complaints_table = gr.Dataframe(
                    value=complaint_df,
                    label="All Complaints",
                    interactive=False
                )

                with gr.Row():
                    complaint_id_update = gr.Textbox(label="Complaint ID")
                    new_status = gr.Dropdown(
                        choices=["Registered", "In Progress", "Resolved", "Closed"],
                        label="New Status"
                    )
                    update_complaint_btn = gr.Button("Update Status", variant="secondary")

                def update_complaint_status(comp_id, status):
                    if not comp_id or not status:
                        return "Please provide both complaint ID and new status.", complaint_df

                    for complaint in complaints:
                        if complaint['id'] == comp_id:
                            complaint['status'] = status
                            complaint['updated_at'] = datetime.now()
                            updated_df = pd.DataFrame(complaints)
                            return f"Complaint {comp_id} status updated to {status}", updated_df

                    return f"Complaint {comp_id} not found.", complaint_df

                update_result = gr.Textbox(label="Update Status", interactive=False)

                update_complaint_btn.click(
                    update_complaint_status,
                    [complaint_id_update, new_status],
                    [update_result, complaints_table]
                )

        # Footer
        gr.HTML("""
        <div style="text-align: center; padding: 20px; margin-top: 30px; border-top: 1px solid #ddd;">
            <p><strong>🏛️ Citizen AI - Intelligent Citizen Engagement Platform</strong></p>
            <p>Powered by IBM Granite 3.0-2B Local AI Model | Built with Gradio</p>
            <p><small>🌟 Empowering citizens through AI-driven governance solutions</small></p>
        </div>
        """)

    return app

# Step 7: Launch the application
def main():
    """Main function to launch the Citizen AI platform"""
    try:
        print("🚀 Starting Citizen AI Platform...")

        # Create the interface
        app = create_main_interface()

        # Launch configuration
        launch_config = {
            "share": True,  # Creates public link for Colab
            "server_name": "0.0.0.0",
            "server_port": 7860,
            "show_tips": True,
            "enable_queue": True,
            "max_threads": 4
        }

        print("✅ Interface created successfully!")
        print("🌐 Launching web interface...")
        print(f"🤖 AI Model Status: {'✅ Ready' if citizen_ai.model_loaded else '❌ Limited functionality'}")

        # Launch the app
        app.launch(**launch_config)

    except Exception as e:
        print(f"❌ Error launching application: {e}")
        print("🔄 Trying with basic configuration...")

        # Fallback launch
        try:
            app = create_main_interface()
            app.launch(share=True)
        except Exception as fallback_error:
            print(f"❌ Fallback launch failed: {fallback_error}")
            print("Please check your environment and try again.")


    # Launch the main application
main()

🔄 Downloading IBM Granite 3.3-2B model...
⏳ This may take a few minutes on first run...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

✅ Model loaded successfully on cuda
🚀 Initializing Citizen AI Platform...
✅ System initialized successfully!
🚀 Starting Citizen AI Platform...


/tmp/ipython-input-1241562814.py:523: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(


✅ Interface created successfully!
🌐 Launching web interface...
🤖 AI Model Status: ✅ Ready
❌ Error launching application: Blocks.launch() got an unexpected keyword argument 'show_tips'
🔄 Trying with basic configuration...


/tmp/ipython-input-1241562814.py:523: UserWarning:

You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1becccfd7d4e3a21a6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
